# Investigate Postgresql quality control tables


For this notebook to run one must be connected to the IoA VPN. You also need a local credentials.py file with the connection credentials.

In [30]:
import psycopg2
import psycopg2 as pg2
from psycopg2 import extras
import sys, os
import numpy as np
import pandas as pd
import pandas.io.sql as psql
from astropy.table import Table
import numpy as np
from os import stat, path

#local file with private PGHOST,PGDATABASE,PGUSER,PGPASSWORD values
import credentials as creds

In [ ]:
## ****** LOAD PSQL DATABASE ***** ##


# Set up a connection to the postgres server.
conn_string = "host="+ creds.PGHOST +" port="+ "5432" +" dbname="+ creds.PGDATABASE +" user=" + creds.PGUSER \
+" password="+ creds.PGPASSWORD
conn=psycopg2.connect(conn_string)
print("Connected!")

# Create a cursor object
cursor = conn.cursor()


def load_data(schema, table):

    sql_command = "SELECT * FROM {}.{};".format(str(schema), str(table))
    print (sql_command)

    # Load the data
    data = pd.read_sql(sql_command, conn)

    print(data.shape)
    return (data)

In [2]:

def query_vista(query_str):
    conn = pg2.connect(
        host=creds.PGHOST, user=creds.PGUSER,
        database=creds.PGDATABASE,
        password=creds.PGPASSWORD
    )
    cur = conn.cursor(cursor_factory=extras.DictCursor)
    
    cur.execute(query_str)
    results = cur.fetchall()
    cur.close()
    conn.close()
    return results

In [ ]:
query_str = """
        SELECT filename,chipno,magzpt
        FROM vistaqc;
    """
query_vista(query_str)

In [ ]:

colour_dict = {'Z':'blue','Y':'green','J':'yellow','H':'orange','Ks':'red'}


In [ ]:
query_str = """
        SELECT *
        FROM vistaqc;
    """
#stack=query_vista(query_str)

In [26]:
conn = pg2.connect(
        host=creds.PGHOST, user=creds.PGUSER,
        database=creds.PGDATABASE,
        password=creds.PGPASSWORD
)
cur = conn.cursor(cursor_factory=extras.DictCursor)
    
cur.execute(query_str)
results = cur.fetchall()
#cur.close()
#conn.close()

In [28]:


conn = pg2.connect(
        host=creds.PGHOST, user=creds.PGUSER,
        database=creds.PGDATABASE,
        password=creds.PGPASSWORD
)
query_str = """
SELECT *
FROM vistaqc
limit 10
    """
dat = pd.read_sql_query(query_str, conn)
conn = None

In [31]:
tab=Table.from_pandas(dat)

In [32]:
tab

image_id,filename,origfile,esoname,obj,obscat,obstech,obstype,filepath,ra,dec,glon,glat,coords,equinox,radecsys,image_exptime,exectime,totexptime,mjd,dateobs,nightobs,utc,lst,picoi,observer,obscat_id,obstech_id,obstype_id,insfilter_id,inspres1,obsdid_id,obsid,obsname,irunno,obspicoiid,obspicoiname,obsstart,obstplno,ocsrecipe,ocsreqtime,amstart,amend,alt,telfwhmstart,telfwhmend,obsairm,obsfwhm,obstrans,obscontainerid,obscontainertype,obsgrp,obsmoondist,obsmoonfli,telapstart,telapend,telrhum,teltau0,teltemp,winddir,windsp,az,parangstart,parangend,moon_ra,moon_dec,posang,tpldid_id,tplexpno,tplid,tplname,tplnexp,tplstart,njitter,noffsets,nustep,obsnum,reqtime,jitterx,jittery,offsetx,offsety,programme_id,semester_id,jittername,offsetname,skyalgo,is_checked,version,is_public,is_tile,is_stack,st_mtime,bitpix,naxis1,naxis2,exptime,extension_mjd,extension_dateobs,ctype1,ctype2,crval1,crval2,crpix1,crpix2,cd1_1,cd1_2,cd2_1,cd2_2,pv2_1,pv2_2,pv2_3,pv2_4,pv2_5,detdid_id,detdit,chiplife,chipno,detframno,detframtype,detmodename,detncorrs,winnx,winny,winstartx,winstarty,expname,expno,runno,mindit,ndit,nditskip,nditcor,cenra,cendec,cencoords,ra1,dec1,ra2,dec2,ra3,dec3,ra4,dec4,dribble,pixsize,nprov,cirvers,saturation,ellipticity,magzpt,magzerr,magznpt,darkcor,flatcor,lincor,skysub,confmap,stripecor,striperms,skylevel,skynoise,stdcrms,numbrms,classifd,threshol,minpix,crowded,rcore,seeing,filtfwhm,extinct,maglim,skybright,ebmvmed,apcorpk,apcor1,apcor2,apcor3,apcor4,apcor5,apcor6,apcor7,nobjects,imcorever,catcoordver,classver,classpver,photompver,extension_id,nightzpt,nightzrr,nightnum,prog,semester,surveyname,groupname,obstatus,qcstatus,filtname,filtid,effwave,sunset,sunrise,dusk,dawn,moonrise,moonset,moonphase,moonaltsunrise,moonaltsunset,moonaltdawn,moonaltdusk,magzp1,magzp2,magzp3,magzp4,magzp5,magzp6,magzp_max,magzp_min,ellpt1,ellpt2,ellpt3,ellpt4,ellpt5,ellpt6,ellpt_max,ellpt_min,seeng1,seeng2,seeng3,seeng4,seeng5,seeng6,seeng_max,seeng_min
int64,str22,str27,str34,str4,str7,str12,str6,str38,float64,float64,float64,float64,str23,float64,str3,float64,float64,float64,float64,object,int64,float64,float64,str7,str7,int64,int64,int64,int64,float64,int64,int64,str9,float64,int64,str7,object,int64,str7,float64,float64,float64,float64,float64,float64,float64,float64,str4,int64,str1,str1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,str20,str25,int64,object,int64,int64,int64,int64,float64,float64,float64,float64,float64,int64,int64,str9,str6,str7,bool,float64,bool,bool,bool,object,int64,int64,int64,float64,float64,object,str8,str8,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,bool,int64,int64,str3,str1,int64,int64,int64,object,object,str22,int64,float64,float64,int64,int64,bool,float64,float64,str23,float64,float64,float64,float64,float64,float64,float64,float64,str8,float64,int64,float64,float64,float64,float64,float64,float64,str21,str20,str17,str23,str27,bool,float64,float64,float64,float64,int64,bool,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,object,object,object,object,object,int64,float64,float64,float64,str10,str1,str3,str8,str9,str1,str2,str5,float64,object,object,object,object,object,object,float64,float64,float64,float64,float64,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
682656,v20120716_00145_st.fit,VIRCAM_IMG_OBS198_0097.fits,VCAM.2012-07-16T23:46:24.3695.fits,b370,SCIENCE,"IMAGE,JITTER",OBJECT,/data/apm44_e/vista/processed/20120716,258.051379,-33.66802,352.10908934,3.32484586206,17:12:12.33 -33:40:04.9,2000.0,FK5,4.0,363.0,8.0,56124.9905599,2012-07-16T23:46:24.000,20120716,85583.0,53139.173,UNKNOWN,UNKNOWN,2,1,3,2,0.0,5,716146,b370v-029,2.012071600145009e+

In [33]:
tab.colnames

['image_id',
 'filename',
 'origfile',
 'esoname',
 'obj',
 'obscat',
 'obstech',
 'obstype',
 'filepath',
 'ra',
 'dec',
 'glon',
 'glat',
 'coords',
 'equinox',
 'radecsys',
 'image_exptime',
 'exectime',
 'totexptime',
 'mjd',
 'dateobs',
 'nightobs',
 'utc',
 'lst',
 'picoi',
 'observer',
 'obscat_id',
 'obstech_id',
 'obstype_id',
 'insfilter_id',
 'inspres1',
 'obsdid_id',
 'obsid',
 'obsname',
 'irunno',
 'obspicoiid',
 'obspicoiname',
 'obsstart',
 'obstplno',
 'ocsrecipe',
 'ocsreqtime',
 'amstart',
 'amend',
 'alt',
 'telfwhmstart',
 'telfwhmend',
 'obsairm',
 'obsfwhm',
 'obstrans',
 'obscontainerid',
 'obscontainertype',
 'obsgrp',
 'obsmoondist',
 'obsmoonfli',
 'telapstart',
 'telapend',
 'telrhum',
 'teltau0',
 'teltemp',
 'winddir',
 'windsp',
 'az',
 'parangstart',
 'parangend',
 'moon_ra',
 'moon_dec',
 'posang',
 'tpldid_id',
 'tplexpno',
 'tplid',
 'tplname',
 'tplnexp',
 'tplstart',
 'njitter',
 'noffsets',
 'nustep',
 'obsnum',
 'reqtime',
 'jitterx',
 'jittery',


In [34]:
conn = pg2.connect(
        host=creds.PGHOST, user=creds.PGUSER,
        database=creds.PGDATABASE,
        password=creds.PGPASSWORD
)
query_str = """
SELECT DISTINCT surveyname
FROM vistaqc
    """
dat = pd.read_sql_query(query_str, conn)
conn = None

In [35]:
dat

,surveyname
0,JVarCar
1,GT980
2,VVVLight
3,VIDEO
4,NBCosmos
5,Simon
6,Calib
7,SV
8,UltraVISTA
9,Orion A


In [36]:
conn = pg2.connect(
        host=creds.PGHOST, user=creds.PGUSER,
        database=creds.PGDATABASE,
        password=creds.PGPASSWORD
)
query_str = """
SELECT COUNT(*)
FROM vistaqc
    """
dat = pd.read_sql_query(query_str, conn)
conn = None

In [37]:
dat

,count
0,11121212


In [40]:
def str_to_astropy(query_str=None):
    conn = pg2.connect(
        host=creds.PGHOST, user=creds.PGUSER,
        database=creds.PGDATABASE,
        password=creds.PGPASSWORD
    )
    if query_str==None:
        query_str = """
        SELECT COUNT(*)
        FROM vistaqc
        """
    dat = pd.read_sql_query(query_str, conn)
    tab=Table.from_pandas(dat)
    conn = None
    return tab


In [ ]:
video = str_to_astropy("SELECT FROM vistaqc WHERE surveyname='VIDEO'")